In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import seaborn as sns
import sys

sys.path.append("..")
from src.visualization.visualize import DrawNBACourt
from src.utils import load_joblib, calc_dists

In [2]:
%matplotlib inline

train_raw_fp = Path("../data") / "raw" / "train.jbl"
train_interim_fp = Path("../data") / "interim" / "train.jbl"

x_length = 47
y_length = 50

In [3]:
train = load_joblib(train_interim_fp)
train

,frame,scr_x,scr_y,usr_x,usr_y,uDF_x,uDF_y,bal_x,bal_y,is_screen_play,filename
0,0,2.89,4.74,5.49,1.50,2.78,5.22,6.98,12.70,1,0000
1,1,2.88,4.70,5.52,1.51,2.80,5.20,7.08,12.52,1,0000
2,2,2.87,4.67,5.54,1.53,2.82,5.19,7.13,12.35,1,0000
3,3,2.86,4.65,5.56,1.54,2.84,5.17,7.08,12.37,1,0000
4,4,2.86,4.61,5.58,1.55,2.86,5.15,7.00,12.44,1,0000
...,...,...,...,...,...,...,...,...,...,...,...
147411,40,9.37,11.86,11.85,12.29,8.84,12.23,11.56,12.25,0,1527
147412,41,9.36,11.85,11.83,12.31,8.84,12.21,11.41,12.58,0,1527
147413,42,9.34,11.84,11.82,12.33,8.83,12.19,11.39,12.62,0,1527
147414,43,9.33,11.84,11.81,12.36,8.82,12.17,11.29,12.58,0,1527


In [5]:
scr_cols = ("scr_x", "scr_y")
usr_cols = ("usr_x", "usr_y")
uDF_cols = ("uDF_x", "uDF_y")
ball_cols = ("bal_x", "bal_y")

train = calc_dists(train, usr_cols, scr_cols)
train = calc_dists(train, usr_cols, uDF_cols)
train = calc_dists(train, scr_cols, uDF_cols)
train = calc_dists(train, usr_cols, ball_cols)
train = calc_dists(train, scr_cols, ball_cols)
train = calc_dists(train, uDF_cols, ball_cols)
train.head()

,frame,scr_x,scr_y,usr_x,usr_y,uDF_x,uDF_y,bal_x,bal_y,is_screen_play,filename,dist_usr_scr,dist_usr_uDF,dist_scr_uDF,dist_usr_bal,dist_scr_bal,dist_uDF_bal
0,0,2.89,4.74,5.49,1.50,2.78,5.22,6.98,12.70,1,0000,4.154227,4.602445,0.492443,11.298677,8.949285,8.578485
1,1,2.88,4.70,5.52,1.51,2.80,5.20,7.08,12.52,1,0000,4.140737,4.584158,0.506360,11.119969,8.876508,8.479434
2,2,2.87,4.67,5.54,1.53,2.82,5.19,7.13,12.35,1,0000,4.121711,4.560044,0.522398,10.936201,8.782369,8.357135
3,3,2.86,4.65,5.56,1.54,2.84,5.17,7.08,12.37,1,0000,4.118507,4.536000,0.520384,10.936146,8.798113,8.355693
4,4,2.86,4.61,5.58,1.55,2.86,5.15,7.00,12.44,1,0000,4.094142,4.512028,0.540000,10.982190,8.857116,8.383537


In [9]:
agg_methods = ["mean", "min", "max", "std"]
train_agg = train.groupby("filename").agg({
    "is_screen_play": ["mean"],
    "dist_usr_scr": agg_methods,
    "dist_usr_uDF": agg_methods,
    "dist_scr_uDF": agg_methods,
    "dist_usr_bal": agg_methods,
    "dist_scr_bal": agg_methods,
    "dist_uDF_bal": agg_methods,
})
train_agg.columns = [f"{col[0]}_{col[1]}" for col in train_agg.columns]
train_agg

,is_screen_play_mean,dist_usr_scr_mean,dist_usr_scr_min,dist_usr_scr_max,dist_usr_scr_std,dist_usr_uDF_mean,dist_usr_uDF_min,dist_usr_uDF_max,dist_usr_uDF_std,dist_scr_uDF_mean,...,dist_usr_bal_max,dist_usr_bal_std,dist_scr_bal_mean,dist_scr_bal_min,dist_scr_bal_max,dist_scr_bal_std,dist_uDF_bal_mean,dist_uDF_bal_min,dist_uDF_bal_max,dist_uDF_bal_std
filename,,,,,,,,,,,,,,,,,,,,,
0000,1,3.440931,1.867753,4.555546,0.960715,3.653044,2.000850,4.867381,1.012476,0.502312,...,11.471900,4.747551,6.443959,1.382208,9.701861,3.282937,6.422878,1.823897,9.315176,3.024244
0001,1,2.639742,1.358823,3.895138,0.799817,1.348713,0.543415,2.584144,0.534599,1.794898,...,10.486110,3.182270,2.443085,0.242074,7.826781,2.603814,4.088809,1.211982,8.184626,2.823689
0002,1,6.374090,2.577208,9.495562,2.714266,3.958844,0.489183,6.061229,1.623740,2.550696,...,11.536767,3.362654,8.366913,6.537920,10.460617,1.351251,7.070959,0.981835,11.968162,3.631720
0003,1,2.929659,0.441475,5.155589,1.519372,1.831413,0.380789,3.166228,0.854128,1.881963,...,7.959510,2.919333,4.027033,0.717844,6.769963,1.837305,2.533231,0.444072,5.748495,1.660464
0004,1,3.557792,0.900278,5.792271,1.506162,2.629966,0.948103,3.601805,0.942569,1.123080,...,6.029743,0.902867,5.905209,3.871046,7.539423,1.036460,5.389925,2.590309,6.922752,1.117004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,0,4.360884,0.841487,6.130008,1.768958,6.148282,1.355655,8.606840,2.599115,2.445693,...,4.672483,1.001470,6.368121,0.300167,9.915090,2.942264,7.847429,0.138924,12.880349,4.269125
1524,0,3.635613,1.947332,7.653947,1.464671,2.996426,0.486004,7.383888,1.696921,1.131848,...,1.075174,0.203232,3.658909,1.696408,7.372706,1.385610,3.023640,0.764199,7.140623,1.595942
1525,0,5.887245,4.490902,7.880634,1.235655,2.796756,0.460977,6.036497,1.434319,3.603346,...,10.106656,1.847365,7.483779,5.214221,9.108853,1.411561,6.150071,4.483626,9.516396,1.364028
